# Thin plate Spline Spatial Transformer Network

## Imports

In [1]:
import argparse
import logging
import time
# select GPU on the server
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'
# pytorch related package 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models

print('pytorch version: ' + torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# math and showcase
import matplotlib.pyplot as plt
import numpy as np

pytorch version: 1.8.1+cu111


In [2]:
# stub for argparse
class opts:
    def __init__(self):
        self.resume = False
        self.sess = 'test_run'
        self.seed = 35328880
        self.batch_size = 64
        self.epochs = 10
        self.n_grid_density = 4
        self.image_size = 28

args = opts()

## Models
follower: neural network of the original task. For now a simple small convolution neural network.

In [3]:
class follower(nn.Module):
    def __init__(self, args):
        super(follower, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

localizer, part of the STN, train on data and output the distortion kernel for TPS

In [4]:
class localizer(nn.Module):
    def __init__(self, args, source_points):
        super(localizer, self).__init__()
        self.args = args
        n_output = args.n_grid_density**2 *2
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, n_output)
        # insert control point as bias
        bias = torch.flatten(source_points)
        self.fc2.bias.data.copy_(bias)
        self.fc2.weight.data.zero_()

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x) # no activation
        return x.view(self.args.batch_size, -1, 2)

tps_warper, a function that warp the image by distortion kernel

In [5]:
class tps_warper(nn.Module):
    def __init__(self, args, source_points):
        super(tps_warper, self).__init__()
        self.args = args
        self.src_points = source_points
        
    def get_tensortype(self, tensor):
        return dict(dtype=tensor.dtype, device=tensor.device)
    
    def pair_square_euclidean(self, x1, x2):
        # ||x1 - x2||^2 = (x1-x2)^T(x1-x2) = x1^T*x1 + x2^T*x2 - 2*x1^T*x2
        x1_sq = x1.mul(x1).sum(dim=-1, keepdim=True)
        x2_sq = x2.mul(x2).sum(dim=-1, keepdim=True).transpose(1,2)
        x1_x2 = x1.matmul(x2.transpose(1,2))
        square_dist = -2 * x1_x2 + x1_sq + x2_sq
        square_dist = square_dist.clamp(min=0)  # handle possible numerical errors
        return square_dist
    def kernel_distance(self, r_sq, eps=1e-8):
        # Compute the TPS kernel distance function: r^2*log(r), where r is the euclidean distance
        return 0.5 * r_sq * r_sq.add(eps).log()
    
    def get_tps_parameters(self, source_points, dest_points):
        tensortype = self.get_tensortype(source_points)
        batch_size, n_points = source_points.shape[:2]
        # TPS warping has a close form equation:
        # find a_1, a_x, a_y, w_i in some equation (a surface for the warping)
        # to get coefficients (a_1, a_x, a_y, w_i), we need to solve a linear system
        # build matrix L
        pair_distance = self.pair_square_euclidean(source_points, dest_points)
        k_matrix = self.kernel_distance(pair_distance)
        dest_with_zeros = torch.cat((dest_points, torch.zeros(batch_size, 3, 2, **tensortype)), 1)
        p_matrix = torch.cat((torch.ones(batch_size, n_points, 1, **tensortype), source_points), -1)
        p_matrix_t = torch.cat((p_matrix, torch.zeros(batch_size, 3, 3, **tensortype)), 1).transpose(1,2)
        l_matrix = torch.cat((k_matrix, p_matrix), -1)
        l_matrix = torch.cat((l_matrix, p_matrix_t), 1)
        # solve the linear system
        weights, _ = torch.solve(dest_with_zeros, l_matrix)
        rbf_weights = weights[:, :-3]
        affine_weights = weights[:, -3:]
        # with these weights, we got a function for TPS warping
        return rbf_weights, affine_weights 
    
    def tps_warp_points(self, source_points, kernel_points, rbf_weights, affine_weights):
        # map all pixel to its new location
        pair_distance = self.pair_square_euclidean(source_points, kernel_points)
        k_matrix = self.kernel_distance(pair_distance)
        # rerpeat on x and y
        k_matrix = k_matrix.unsqueeze(3).repeat(1,1,1,2)
        source_points = source_points.unsqueeze(3).repeat(1,1,1,2)
        # add pixel dimension for broadcasting
        rbf_weights, affine_weights = rbf_weights.unsqueeze(1), affine_weights.unsqueeze(1)

        warped = k_matrix.mul(rbf_weights).sum(-2) + \
                 source_points.mul(affine_weights[:,:,1:,:]).sum(-2) + \
                 affine_weights[:,:,0,:]
        return warped
    
    def tps_warp_image(self, image, kernel_points, rbf_weights, affine_weights):
        tensortype = self.get_tensortype(kernel_points)
        batch_size, _, h, w = image.shape
        # create the grid to represent all the pixel
        ys, xs = torch.meshgrid(torch.linspace(-1, 1, h, **tensortype),
                                torch.linspace(-1, 1, w, **tensortype))
        coords = torch.stack([xs, ys], -1).view(-1, 2)
        coords = torch.stack([coords]*batch_size, 0)
        warped = self.tps_warp_points(coords, kernel_points, rbf_weights, affine_weights).view(-1, h, w, 2)
        warped_image = F.grid_sample(image, warped, align_corners=False)
        return warped_image

    def forward(self, image, dst_points):
        rbf_weights, affine_weights = self.get_tps_parameters(dst_points, self.src_points)
        warped_image = self.tps_warp_image(image, self.src_points, rbf_weights, affine_weights)
        return warped_image

tps_stn, the structure of the whole tps-stn

In [6]:
class tps_stn(nn.Module):
    def __init__(self, args):
        super(tps_stn, self).__init__()
        self.args = args
        src_points = self.get_src_points(args.batch_size, args.n_grid_density).to(device)
        self.src_points = src_points.unsqueeze(0).repeat(args.batch_size, 1, 1)

        self.follower = follower(args)
        self.localizer = localizer(args, src_points)
        self.tps_warper = tps_warper(args, self.src_points)

    # src_points create upon the n_grid_density is given
    def get_src_points(self, batch_size, n_grid_density=4, grid_span=0.9):
        src_points_1d = torch.linspace(-grid_span, grid_span, steps=n_grid_density)
        src_points_2d = torch.cartesian_prod(src_points_1d, src_points_1d)
        return src_points_2d
        
    def stn(self, x):
        kernel_points = self.localizer(x)
        warped_x = self.tps_warper(x, kernel_points)
        return warped_x
        
    def forward(self, x):
        warped_x = self.stn(x)
        x = self.follower(warped_x)
        return x

In [7]:
def main():
    args = opts() # TODO: replace it by argparse
    # Set seeds
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)

    print('==> Preparing dataset..')
    # Training dataset
    mnist_train = datasets.MNIST("./data/MNIST", train=True, download=True, transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
    train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=args.batch_size, shuffle=True, drop_last=True, num_workers=2)
    # Test dataset
    mnist_test = datasets.MNIST("./data/MNIST", train=True, download=True, transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
    test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=args.batch_size, shuffle=True, drop_last=True, num_workers=2)

    # Load model
    if args.resume:
        # Load checkpoint.
        print('==> Resuming from checkpoint..')
        model = tps_stn(args).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

        checkpoint = torch.load('./checkpoint/' + args.sess + '_' + str(args.seed) + '.pth')
        prev_acc = checkpoint['acc']
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch_start = checkpoint['epoch'] + 1
        torch.set_rng_state(checkpoint['rng_state'])
    else:
        print('==> Building model..')
        epoch_start = 0
        prev_acc = 0.0
        model = tps_stn(args).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # Logger
    result_folder = './results/'
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    logger = logging.getLogger(__name__)
    logname = model.__class__.__name__ + '_' + args.sess + \
        '_' + str(args.seed) + '.log'
    logfile = os.path.join(result_folder, logname)
    if os.path.exists(logfile):
        os.remove(logfile)
    logging.basicConfig(
        format='[%(asctime)s] - %(message)s',
        datefmt='%Y/%m/%d %H:%M:%S',
        level=logging.INFO,
        filename=logfile
    )
    logger.info(args)

    # Training
    def train(epoch):
        print('\nEpoch: {:04}'.format(epoch))
        train_loss, correct, total = 0, 0, 0
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output_logit = model(data)
            loss = F.nll_loss(output_logit, target)
            loss.backward()
            optimizer.step()
            preds = F.softmax(output_logit, dim=1)
            preds_top_p, preds_top_class = preds.topk(1, dim=1)

            train_loss += loss.item() * target.size(0)
            total += target.size(0)
            correct += (preds_top_class.view(target.shape) == target).sum().item()

        return (train_loss / batch_idx, 100. * correct / total)

    # Test
    def test(epoch):
        model.eval()
        test_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader):
                data, target = data.to(device), target.to(device)
    
                optimizer.zero_grad()
                output_logit = model(data)
                loss = F.nll_loss(output_logit, target)
                preds = F.softmax(output_logit, dim=1)
                preds_top_p, preds_top_class = preds.topk(1, dim=1)
    
                test_loss += loss.item() * target.size(0)
                total += target.size(0)
                correct += (preds_top_class.view(target.shape) == target).sum().item()
        
        return (test_loss / batch_idx, 100. * correct / total)
            
    # Save checkpoint
    def checkpoint(acc, epoch):
        print('==> Saving..')
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        save_path = './checkpoint/' + args.sess + '_' + str(args.seed) + '.pth'
        torch.save({
            'epoch': epoch,
            'acc': acc,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'rng_state': torch.get_rng_state(),
            }, save_path)
    
    # Run
    logger.info('Epoch \t Seconds \t \t Train Loss \t Train Acc')
    start_train_time = time.time()
    for epoch in range(epoch_start, args.epochs):
        start_epoch_time = time.time()
        
        train_loss, train_acc = train(epoch)
        epoch_time = time.time()
        logger.info('%5d \t %7.1f \t \t %10.4f \t %9.4f',
            epoch, epoch_time - start_epoch_time, train_loss, train_acc)
        # Save checkpoint.
        if train_acc - prev_acc  > 0.1:
            prev_acc = train_acc
            checkpoint(train_acc, epoch)
    train_time = time.time()
    logger.info('Total train time: %.4f minutes', (train_time - start_train_time)/60)

    # Evaluation
    logger.info('Test Loss \t Test Acc')
    test_loss, test_acc = test(epoch)
    logger.info('%9.4f \t %8.4f', test_loss, test_acc)
    
main()

==> Preparing dataset..
==> Building model..

Epoch: 0000
==> Saving..

Epoch: 0001

Epoch: 0002

Epoch: 0003

Epoch: 0004

Epoch: 0005

Epoch: 0006

Epoch: 0007

Epoch: 0008

Epoch: 0009
==> Saving..
